In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [ ]:
#links
estados = ['https://www.conicyt.cl/blog/category/concursos/abiertos/', 'https://www.conicyt.cl/blog/category/concursos/en-evaluacion/', 'https://www.conicyt.cl/blog/category/concursos/adjudicados/', 'https://www.conicyt.cl/blog/category/concursos/proximos/']


def links_proyectos(estados_chile):
    est = pd.DataFrame()
    links = []
    cant_links_est = []
    for estado in estados:
        
        parserT = parser(estado)

        #Extrae y organiza lista de paginacion
        paginacion = parserT.xpath('//div[@class="pagenavi"]/ul//a/@href')
        
        urls = []
        pagina = estado
        if len(paginacion) != 0:
            paginacion.pop()
            paginacion.append('?pag=1')
            for link in paginacion:
                urls.append(pagina+link)
        else:
            urls.append(estado)
        

        

        #Extrae links de cada pag
        n = 0
        for url in urls:
            parser_url = parser(url)
            links_pagina = parser_url.xpath('//div[@id="contenido_doble"]//h2/a/@href')
            links += links_pagina
            n += len(links_pagina)


        cant_links_est.append(n)

    estados_proy = []
    for i in range(cant_links_est[0]):
        estados_proy.append('Abierto')
    for i in range(cant_links_est[1]):
        estados_proy.append('En evaluación')
    for i in range(cant_links_est[2]):
        estados_proy.append('Cerrados')
    for i in range(cant_links_est[3]):
        estados_proy.append('Proximos')
    


    est['Estado'] = estados_proy
    est['Links'] = links

    indexNames = est[ est['Links'].str.startswith('https://www.conicyt.cl/becasconicyt') ].index
    # Delete these row indexes from dataFrame
    est.drop(indexNames, inplace=True)



    return est

urls_proyectos = links_proyectos(estados)
urls_proyectos

In [ ]:
split_url_proyectos = []
for i in range(0, len(urls_proyectos['Links']), 10):
    split_url_proyectos.append(urls_proyectos['Links'][i:i+10]) 
split_url_proyectos[0]

In [74]:
### avance chile
chile = pd.DataFrame()


descripciones = []
publicos = []
bitacoras = []
links = []
titulos = []

descripcion = ''
p_objetivo = ''
bitacora = ''
titulo = ''
link = ''
n=0
while True:
    chile_sub = pd.DataFrame()
    
    for url_proy in split_url_proyectos[0][7::]:
        print(url_proy)
        driver = webdriver.Chrome()
        sleep(random.randrange(5, 8))
        driver.get(url_proy)
        
        #Saca Titulo
                
        try:
            titulo = driver.find_element_by_xpath('//h2[@id="title-concurso"]').text
            titulos.append(titulo)    
        except:
            if len(titulo) == 0:
                titulo = driver.find_element_by_xpath('//h2[@class="title_content"]').text
                titulos.append(titulo)
            else:
                titulo = ''
                titulos.append(titulo) 
        
        #try:
        #    titulo = driver.find_element_by_xpath('//h2[@id="title-concurso"]').text
        #    titulos.append(titulo)
        #except:
        #        
        #    try:
        #        titulo = driver.find_element_by_xpath('//h2[@class="title_content"]').text
        #        titulos.append(titulo)
        #    except:        
        #        titulo = ''
        #        titulos.append(titulo)


        ### Saca el formato descripción

        try:
            boton_descripcion = driver.find_element_by_xpath('//div[contains(text(), "presentación")]')
                    
        except:
            try:
                boton_descripcion = driver.find_element_by_xpath('//li[contains(text(), "Presentación")]')
            except:
                try:
                    boton_descripcion = driver.find_element_by_xpath('//div[contains(@class, "accordion-Presentacin")]')
                except:
                    descripcion = ''
                    descripciones.append(descripcion)
                    
        # ---                
        try:              
            boton_descripcion.click()

            try:
                descripcion = driver.find_element_by_xpath('//div[@id="content-1"]').text
                descripciones.append(descripcion)
            except:
                try:
                    descripcion = driver.find_element_by_xpath('//div[@id="pane-0-0"]').text
                    descripciones.append(descripcion)
                except:
                    try:
                        descripcion = driver.find_element_by_xpath('//div[@class="arconix-accordion-content"]').text
                        descripciones.append(descripcion)
                    except:
                        descripcion = ''
                        descripciones.append(descripcion)
        except:
            descripcion = ''

        ### Saca el publico objetivo
        
        try:
            boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "público")]')
                
        except:
            try:
                boton_p_objetivo = driver.find_element_by_xpath('//div[contains(text(), "Público")]')
            except:
                try:
                    boton_p_objetivo = driver.find_element_by_xpath('//ul//li[contains(text(), "Público")]') 
                except:
                    p_objetivo = ''
                    publicos.append(p_objetivo)
        #---            
        try:                            
            boton_p_objetivo.click()
            try:
                p_objetivo = driver.find_element_by_xpath('.//div[@id="content-2"]').text
                publicos.append(p_objetivo)
            except:
                try:
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text    
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-2"]').text    
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-3"]').text
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-4"]').text
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-5"]').text
                    if len(p_objetivo) == 0:
                        p_objetivo = driver.find_element_by_xpath('.//div[@id="pane-0-6"]').text
                    publicos.append(p_objetivo)
                except:
                    p_objetivo = ''
                    publicos.append(p_objetivo)
        except NameError:
            p_objetivo = ''
                

            
        ### Saca Bitacoras
        try:
            boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitacora")]')
        except:
            try:
                boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "bitácora")]')
            except:
                try:
                    boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitácora")]')
                except:
                    try:
                        boton_bitacora = driver.find_element_by_xpath('//div[contains(text(), "Bitacora")]')
                    except:
                        try:
                            boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "bitacora")]') 
                        except:
                            try:
                                boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "Bitácora")]')
                            except:
                                try:
                                    boton_bitacora = driver.find_element_by_xpath('//li[contains(text(), "Bitacora")]')
                                except:
                                    bitacora = ''
                                    bitacoras.append(bitacora)
                            
        #---                                        
        try:
            boton_bitacora.click()    
            try:
                bitacora = driver.find_element_by_xpath('.//div[@id="content-3"]').text
                bitacoras.append(bitacora)
            except:
                    
                try:
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-1"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-2"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-3"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-4"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-5"]').text
                    if len(bitacora) == 0:
                        bitacora = driver.find_element_by_xpath('.//div[@id="pane-0-6"]').text
                    bitacoras.append(bitacora)
                except:
                    bitacora = ''
                    bitacoras.append(bitacora)
        except:
            bitacora = ''

        ### Extrae los links
        try:
            url = driver.find_elements_by_xpath('//div[@class="collapse show"]/div/ul//li/a')

            urls = []
            for r in url:
                url2 = r.get_attribute('href')
                urls.append(url2)

            if len(urls) == 0:
                url = driver.find_elements_by_xpath('//div[@class="filtros"]/ul//li/a')

                urls = []
                for r in url:
                    url2 = r.get_attribute('href')
                    urls.append(url2)

            urls2 = ''
            for url2 in urls:
                urls2 = urls2 + ', ' + url2
            links.append(urls2)
        except:
            urls2 = ''
            links.append(urls2)


        driver.quit()
        
        print(len(titulos))
        print(len(descripciones))
        print(len(publicos))
        print(len(bitacoras))
        print(len(links))
        print(len(split_url_proyectos[n]))
  
            
    if len(titulos)==len(descripciones) and len(descripciones)==len(split_url_proyectos[n]) and len(publicos)==len(split_url_proyectos[n]) and len(bitacoras)==len(split_url_proyectos[n]) and len(links)==len(split_url_proyectos[n]):
        chile_sub['Título']= titulos
        chile_sub['Descripción']=descripciones
        chile_sub['Público Objetivo']=publicos
        chile_sub['Bitacora']=bitacoras
        chile_sub['Link proyecto']=split_url_proyectos[n]
        chile_sub['Pdfs']= links

        chile = chile.append(chile_sub, ignore_index=True)
        n+=1

    else:
        n=n

    #print(chile)
    
    if n == 81:
        break

https://www.conicyt.cl/fondef/2018/08/01/segunda-etapa-proyectos-idea-2016/


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=34207): Max retries exceeded with url: /session/5dc46e81c1d2cf75d15dc4578f1df4a8/element/38175812-659c-4b64-9d09-3649ba431675/click (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f466c417ca0>: Failed to establish a new connection: [Errno 111] Connection refused'))